In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('display.max_columns', 30)

## Tidying variable values as column names with stack
#### This particular messy dataset contains variable values as column names. We will need to transpose these column names into column values. In this recipe, we use the stack method to restructure our DataFrame into tidy form.

In [3]:
state_fruit = pd.read_csv('master/data/state_fruit.csv', index_col=0)
state_fruit

,Apple,Orange,Banana
Texas,12,10,40
Arizona,9,7,12
Florida,0,14,190


In [4]:
state_fruit.stack()

Texas    Apple      12
         Orange     10
         Banana     40
Arizona  Apple       9
         Orange      7
         Banana     12
Florida  Apple       0
         Orange     14
         Banana    190
dtype: int64

In [5]:
# Notice that we now have a Series with a MultiIndex. Let's use the reset_index method to turn the result into a DataFrame:

state_fruit_tidy = state_fruit.stack().reset_index()
state_fruit_tidy

,level_0,level_1,0
0,Texas,Apple,12
1,Texas,Orange,10
2,Texas,Banana,40
3,Arizona,Apple,9
4,Arizona,Orange,7
5,Arizona,Banana,12
6,Florida,Apple,0
7,Florida,Orange,14
8,Florida,Banana,190


In [6]:
type(state_fruit_tidy.columns)

pandas.core.indexes.base.Index

In [7]:
state_fruit_tidy.columns

Index(['level_0', 'level_1', 0], dtype='object')

In [8]:
state_fruit_tidy.columns = ['state', 'fruit', 'weight']
state_fruit_tidy

,state,fruit,weight
0,Texas,Apple,12
1,Texas,Orange,10
2,Texas,Banana,40
3,Arizona,Apple,9
4,Arizona,Orange,7
5,Arizona,Banana,12
6,Florida,Apple,0
7,Florida,Orange,14
8,Florida,Banana,190


In [9]:
# Instead of directly changing the columns attribute like above, it's also possible to use the lesser-known
# Series method rename_axis to set the names of the index levels before using reset_index:

state_fruit.stack() \
           .rename_axis(['state', 'fruit'])

state    fruit 
Texas    Apple      12
         Orange     10
         Banana     40
Arizona  Apple       9
         Orange      7
         Banana     12
Florida  Apple       0
         Orange     14
         Banana    190
dtype: int64

In [10]:
# The name parameter of reset_index() allows to set the column name to use for the column containing the original Series values.

state_fruit.stack() \
           .rename_axis(['state', 'fruit']) \
           .reset_index(name='weight')

,state,fruit,weight
0,Texas,Apple,12
1,Texas,Orange,10
2,Texas,Banana,40
3,Arizona,Apple,9
4,Arizona,Orange,7
5,Arizona,Banana,12
6,Florida,Apple,0
7,Florida,Orange,14
8,Florida,Banana,190


In [11]:
state_fruit2 = pd.read_csv('master/data/state_fruit2.csv')
state_fruit2

,State,Apple,Orange,Banana
0,Texas,12,10,40
1,Arizona,9,7,12
2,Florida,0,14,190


In [12]:
# If we don't set our first column as index, we will have an undesired result:

state_fruit2.stack()

0  State       Texas
   Apple          12
   Orange         10
   Banana         40
1  State     Arizona
   Apple           9
   Orange          7
   Banana         12
2  State     Florida
   Apple           0
   Orange         14
   Banana        190
dtype: object

In [13]:
state_fruit2.set_index('State').stack()

State          
Texas    Apple      12
         Orange     10
         Banana     40
Arizona  Apple       9
         Orange      7
         Banana     12
Florida  Apple       0
         Orange     14
         Banana    190
dtype: int64

## Tidying variable values as column names with melt
#### In this recipe, we use the melt method to tidy a simple DataFrame with variable values as column names.

In [14]:
state_fruit2 = pd.read_csv('master/data/state_fruit2.csv')
state_fruit2

,State,Apple,Orange,Banana
0,Texas,12,10,40
1,Arizona,9,7,12
2,Florida,0,14,190


In [15]:
state_fruit2.melt(id_vars=['State'],
                  value_vars=['Apple','Orange','Banana'],
                  var_name='Fruit',
                  value_name='Weight')

,State,Fruit,Weight
0,Texas,Apple,12
1,Arizona,Apple,9
2,Florida,Apple,0
3,Texas,Orange,10
4,Arizona,Orange,7
5,Florida,Orange,14
6,Texas,Banana,40
7,Arizona,Banana,12
8,Florida,Banana,190


In [16]:
# All the parameters for the melt method are optional, and if you desire all your values to be in a single column
# and their old column labels to be in the other, you may call melt with just its defaults:

state_fruit2.melt()

,variable,value
0,State,Texas
1,State,Arizona
2,State,Florida
3,Apple,12
4,Apple,9
5,Apple,0
6,Orange,10
7,Orange,7
8,Orange,14
9,Banana,40


In [17]:
# same result as in step 2

state_fruit2.melt(id_vars='State')

,State,variable,value
0,Texas,Apple,12
1,Arizona,Apple,9
2,Florida,Apple,0
3,Texas,Orange,10
4,Arizona,Orange,7
5,Florida,Orange,14
6,Texas,Banana,40
7,Arizona,Banana,12
8,Florida,Banana,190


## Stacking multiple groups of variables simultaneously
#### In this recipe, we will tidy our actor DataFrame by simultaneously stacking the actor names and their corresponding Facebook likes with the wide_to_long function.

In [18]:
movie = pd.read_csv('master/data/movie.csv')
actor = movie[['movie_title', 'actor_1_name',
               'actor_2_name', 'actor_3_name',
               'actor_1_facebook_likes',
               'actor_2_facebook_likes',
               'actor_3_facebook_likes']]
actor.head()

,movie_title,actor_1_name,actor_2_name,actor_3_name,actor_1_facebook_likes,actor_2_facebook_likes,actor_3_facebook_likes
0,Avatar,CCH Pounder,Joel David Moore,Wes Studi,1000.0,936.0,855.0
1,Pirates of the Caribbean: At World's End,Johnny Depp,Orlando Bloom,Jack Davenport,40000.0,5000.0,1000.0
2,Spectre,Christoph Waltz,Rory Kinnear,Stephanie Sigman,11000.0,393.0,161.0
3,The Dark Knight Rises,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,27000.0,23000.0,23000.0
4,Star Wars: Episode VII - The Force Awakens,Doug Walker,Rob Walker,NaN,131.0,12.0,NaN


In [19]:
# If we define our variables as the title of the movie, the actor name, and the number of Facebook likes,
# it's impossibleto use a single call to stack or melt.

def change_col_name(col_name):
    col_name = col_name.replace('_name', '')
    if 'facebook' in col_name:
        fb_idx = col_name.find('facebook')
        col_name = col_name[:5] + col_name[fb_idx - 1:] \
                                + col_name[5:fb_idx-1]
    return col_name

# string.replace()  : remplace le 1era rg par le 2eme arg
# string.find()     : donne la position de l'arg

In [20]:
# The rename method accepts many different types of arguments, one of which is a function.
# When passing it to a function, every column name gets implicitly passed to it one at a time.

actor2 = actor.rename(columns=change_col_name)
actor2.head()

,movie_title,actor_1,actor_2,actor_3,actor_facebook_likes_1,actor_facebook_likes_2,actor_facebook_likes_3
0,Avatar,CCH Pounder,Joel David Moore,Wes Studi,1000.0,936.0,855.0
1,Pirates of the Caribbean: At World's End,Johnny Depp,Orlando Bloom,Jack Davenport,40000.0,5000.0,1000.0
2,Spectre,Christoph Waltz,Rory Kinnear,Stephanie Sigman,11000.0,393.0,161.0
3,The Dark Knight Rises,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,27000.0,23000.0,23000.0
4,Star Wars: Episode VII - The Force Awakens,Doug Walker,Rob Walker,NaN,131.0,12.0,NaN


In [21]:
stubs = ['actor', 'actor_facebook_likes']
actor2_tidy = pd.wide_to_long(actor2,
                              stubnames=stubs,
                              i=['movie_title'],
                              j='actor_num',
                              sep='_')
actor2_tidy.head()           

,,actor,actor_facebook_likes
movie_title,actor_num,,
Avatar,1,CCH Pounder,1000.0
Pirates of the Caribbean: At World's End,1,Johnny Depp,40000.0
Spectre,1,Christoph Waltz,11000.0
The Dark Knight Rises,1,Tom Hardy,27000.0
Star Wars: Episode VII - The Force Awakens,1,Doug Walker,131.0


In [22]:
# TP
actor2_tidy.query('movie_title == "The Dark Knight Rises"')      

actor  actor_facebook_likes
movie_title           actor_num                                            
The Dark Knight Rises 1                     Tom Hardy               27000.0
                      2                Christian Bale               23000.0
                      3          Joseph Gordon-Levitt               23000.0

In [23]:
df = pd.read_csv('master/data/stackme.csv')
df

,State,Country,a1,b2,Test,d,e
0,TX,US,0.45,0.3,Test1,2,6
1,MA,US,0.03,1.2,Test2,9,7
2,ON,CAN,0.70,4.2,Test3,4,2


In [24]:
df2 = df.rename(columns = {'a1':'group1_a1', 'b2':'group1_b2',
                           'd':'group2_a1', 'e':'group2_b2'})
df2

,State,Country,group1_a1,group1_b2,Test,group2_a1,group2_b2
0,TX,US,0.45,0.3,Test1,2,6
1,MA,US,0.03,1.2,Test2,9,7
2,ON,CAN,0.70,4.2,Test3,4,2


In [25]:
pd.wide_to_long(df2,
                stubnames=['group1', 'group2'],
                i=['State', 'Country', 'Test'],
                j='Label',
                suffix='.+',
                sep='_')

group1  group2
State Country Test  Label                
TX    US      Test1 a1       0.45       2
                    b2       0.30       6
MA    US      Test2 a1       0.03       9
                    b2       1.20       7
ON    CAN     Test3 a1       0.70       4
                    b2       4.20       2

## Inverting stacked data
#### In this recipe, we will stack/melt a dataset and promptly invert the operation with unstack/pivot back to its original form.

In [26]:
usecol_func = lambda x: 'UGDS_' in x or x == 'INSTNM'
college = pd.read_csv('master/data/college.csv',
                      index_col='INSTNM',
                      usecols=usecol_func)
college.head()

,UGDS_WHITE,UGDS_BLACK,UGDS_HISP,UGDS_ASIAN,UGDS_AIAN,UGDS_NHPI,UGDS_2MOR,UGDS_NRA,UGDS_UNKN
INSTNM,,,,,,,,,
Alabama A & M University,0.0333,0.9353,0.0055,0.0019,0.0024,0.0019,0.0000,0.0059,0.0138
University of Alabama at Birmingham,0.5922,0.2600,0.0283,0.0518,0.0022,0.0007,0.0368,0.0179,0.0100
Amridge University,0.2990,0.4192,0.0069,0.0034,0.0000,0.0000,0.0000,0.0000,0.2715
University of Alabama in Huntsville,0.6988,0.1255,0.0382,0.0376,0.0143,0.0002,0.0172,0.0332,0.0350
Alabama State University,0.0158,0.9208,0.0121,0.0019,0.0010,0.0006,0.0098,0.0243,0.0137


In [27]:
college_stacked = college.stack()
college_stacked.head(18)

INSTNM                                         
Alabama A & M University             UGDS_WHITE    0.0333
                                     UGDS_BLACK    0.9353
                                     UGDS_HISP     0.0055
                                     UGDS_ASIAN    0.0019
                                     UGDS_AIAN     0.0024
                                     UGDS_NHPI     0.0019
                                     UGDS_2MOR     0.0000
                                     UGDS_NRA      0.0059
                                     UGDS_UNKN     0.0138
University of Alabama at Birmingham  UGDS_WHITE    0.5922
                                     UGDS_BLACK    0.2600
                                     UGDS_HISP     0.0283
                                     UGDS_ASIAN    0.0518
                                     UGDS_AIAN     0.0022
                                     UGDS_NHPI     0.0007
                                     UGDS_2MOR     0.0368
                        

In [28]:
college_stacked.unstack()

,UGDS_WHITE,UGDS_BLACK,UGDS_HISP,UGDS_ASIAN,UGDS_AIAN,UGDS_NHPI,UGDS_2MOR,UGDS_NRA,UGDS_UNKN
INSTNM,,,,,,,,,
Alabama A & M University,0.0333,0.9353,0.0055,0.0019,0.0024,0.0019,0.0000,0.0059,0.0138
University of Alabama at Birmingham,0.5922,0.2600,0.0283,0.0518,0.0022,0.0007,0.0368,0.0179,0.0100
Amridge University,0.2990,0.4192,0.0069,0.0034,0.0000,0.0000,0.0000,0.0000,0.2715
University of Alabama in Huntsville,0.6988,0.1255,0.0382,0.0376,0.0143,0.0002,0.0172,0.0332,0.0350
Alabama State University,0.0158,0.9208,0.0121,0.0019,0.0010,0.0006,0.0098,0.0243,0.0137
The University of Alabama,0.7825,0.1119,0.0348,0.0106,0.0038,0.0009,0.0261,0.0268,0.0026
Central Alabama Community College,0.7255,0.2613,0.0044,0.0025,0.0044,0.0000,0.0000,0.0000,0.0019
Athens State University,0.7823,0.1200,0.0191,0.0053,0.0157,0.0010,0.0174,0.0057,0.0334
Auburn University at Montgomery,0.5328,0.3376,0.0074,0.0221,0.0044,0.0016,0.0297,0.0397,0.0246


In [29]:
college2 = pd.read_csv('master/data/college.csv',
                      usecols=usecol_func)
college2.head()

,INSTNM,UGDS_WHITE,UGDS_BLACK,UGDS_HISP,UGDS_ASIAN,UGDS_AIAN,UGDS_NHPI,UGDS_2MOR,UGDS_NRA,UGDS_UNKN
0,Alabama A & M University,0.0333,0.9353,0.0055,0.0019,0.0024,0.0019,0.0000,0.0059,0.0138
1,University of Alabama at Birmingham,0.5922,0.2600,0.0283,0.0518,0.0022,0.0007,0.0368,0.0179,0.0100
2,Amridge University,0.2990,0.4192,0.0069,0.0034,0.0000,0.0000,0.0000,0.0000,0.2715
3,University of Alabama in Huntsville,0.6988,0.1255,0.0382,0.0376,0.0143,0.0002,0.0172,0.0332,0.0350
4,Alabama State University,0.0158,0.9208,0.0121,0.0019,0.0010,0.0006,0.0098,0.0243,0.0137


In [30]:
college_melted = college2.melt(id_vars='INSTNM',
                               var_name='Race',
                               value_name='Percentage')
college_melted.head()

,INSTNM,Race,Percentage
0,Alabama A & M University,UGDS_WHITE,0.0333
1,University of Alabama at Birmingham,UGDS_WHITE,0.5922
2,Amridge University,UGDS_WHITE,0.2990
3,University of Alabama in Huntsville,UGDS_WHITE,0.6988
4,Alabama State University,UGDS_WHITE,0.0158


In [31]:
melted_inv = college_melted.pivot(index='INSTNM',
                                  columns='Race',
                                  values='Percentage')
melted_inv.head()

Race,UGDS_2MOR,UGDS_AIAN,UGDS_ASIAN,UGDS_BLACK,UGDS_HISP,UGDS_NHPI,UGDS_NRA,UGDS_UNKN,UGDS_WHITE
INSTNM,,,,,,,,,
A & W Healthcare Educators,0.0000,0.0,0.0000,0.9750,0.0250,0.0,0.0000,0.0000,0.0000
A T Still University of Health Sciences,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ABC Beauty Academy,0.0000,0.0,0.9333,0.0333,0.0333,0.0,0.0000,0.0000,0.0000
ABC Beauty College Inc,0.0000,0.0,0.0000,0.6579,0.0526,0.0,0.0000,0.0000,0.2895
AI Miami International University of Art and Design,0.0018,0.0,0.0018,0.0198,0.4773,0.0,0.0025,0.4644,0.0324


In [32]:
college2_replication = melted_inv.loc[college2['INSTNM'],
                                      college2.columns[1:]]\
                                 .reset_index()
college2.equals(college2_replication)

True

In [33]:
college.stack().unstack(0)

INSTNM,Alabama A & M University,University of Alabama at Birmingham,Amridge University,University of Alabama in Huntsville,Alabama State University,The University of Alabama,Central Alabama Community College,Athens State University,Auburn University at Montgomery,Auburn University,Birmingham Southern College,Chattahoochee Valley Community College,Concordia College Alabama,South University-Montgomery,Enterprise State Community College,...,New Horizons Medical Institute-Winder,SAE Institute of Technology-Chicago,Beau Monde College of Hair Design-Beau Monde Academy of Cosmetology,Fayette Beauty Academy-Ritz Beauty Academy,Fountain of Youth Academy of Cosmetology,MCI Institute of Technology-Boca Raton,West Coast University-Miami,National American University-Houston,Aparicio-Levy Technical College,Fred D. Learey Technical College,Hollywood Institute of Beauty Careers-West Palm Beach,Hollywood Institute of Beauty Careers-Casselberry,Coachella Valley Beauty College-Beaumont,Dewey University-Mayaguez,Coastal Pines Technical College
UGDS_WHITE,0.0333,0.5922,0.2990,0.6988,0.0158,0.7825,0.7255,0.7823,0.5328,0.8507,0.7983,0.4661,0.0280,0.3046,0.6408,...,0.4175,0.2797,0.64,0.0,1.0,0.0199,0.1522,0.1858,0.2431,0.3731,0.2182,0.1200,0.3284,0.0,0.6762
UGDS_BLACK,0.9353,0.2600,0.4192,0.1255,0.9208,0.1119,0.2613,0.1200,0.3376,0.0704,0.1102,0.4372,0.8758,0.6054,0.2435,...,0.5000,0.4576,0.08,1.0,0.0,0.2815,0.1739,0.6443,0.1215,0.1388,0.4182,0.3333,0.1045,0.0,0.2508
UGDS_HISP,0.0055,0.0283,0.0069,0.0382,0.0121,0.0348,0.0044,0.0191,0.0074,0.0248,0.0195,0.0492,0.0373,0.0153,0.0509,...,0.0388,0.1864,0.20,0.0,0.0,0.6854,0.6087,0.0672,0.6243,0.3080,0.2364,0.4400,0.4925,1.0,0.0359
UGDS_ASIAN,0.0019,0.0518,0.0034,0.0376,0.0019,0.0106,0.0025,0.0053,0.0221,0.0227,0.0517,0.0127,0.0093,0.0153,0.0202,...,0.0194,0.0085,0.08,0.0,0.0,0.0132,0.0217,0.0079,0.0055,0.0000,0.0182,0.0000,0.0149,0.0,0.0045
UGDS_AIAN,0.0024,0.0022,0.0000,0.0143,0.0010,0.0038,0.0044,0.0157,0.0044,0.0074,0.0102,0.0023,0.0000,0.0153,0.0081,...,0.0000,0.0254,0.00,0.0,0.0,0.0000,0.0000,0.0079,0.0055,0.0000,0.0000,0.0000,0.0299,0.0,0.0034
UGDS_NHPI,0.0019,0.0007,0.0000,0.0002,0.0006,0.0009,0.0000,0.0010,0.0016,0.0000,0.0000,0.0035,0.0000,0.0096,0.0029,...,0.0000,0.0000,0.00,0.0,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0149,0.0,0.0017
UGDS_2MOR,0.0000,0.0368,0.0000,0.0172,0.0098,0.0261,0.0000,0.0174,0.0297,0.0000,0.0051,0.0151,0.0031,0.0000,0.0254,...,0.0049,0.0000,0.00,0.0,0.0,0.0000,0.0435,0.0751,0.0000,0.0022,0.0000,0.0400,0.0149,0.0,0.0191
UGDS_NRA,0.0059,0.0179,0.0000,0.0332,0.0243,0.0268,0.0000,0.0057,0.0397,0.0100,0.0000,0.0000,0.0466,0.0019,0.0012,...,0.0000,0.0085,0.00,0.0,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0182,0.0000,0.0000,0.0,0.0028
UGDS_UNKN,0.0138,0.0100,0.2715,0.0350,0.0137,0.0026,0.0019,0.0334,0.0246,0.0140,0.0051,0.0139,0.0000,0.0326,0.0069,...,0.0194,0.0339,0.00,0.0,0.0,0.0000,0.0000,0.0119,0.0000,0.1779,0.0909,0.0667,0.0000,0.0,0.0056


In [34]:
college.T
college.transpose()

INSTNM,Alabama A & M University,University of Alabama at Birmingham,Amridge University,University of Alabama in Huntsville,Alabama State University,The University of Alabama,Central Alabama Community College,Athens State University,Auburn University at Montgomery,Auburn University,Birmingham Southern College,Chattahoochee Valley Community College,Concordia College Alabama,South University-Montgomery,Enterprise State Community College,...,Strayer University-Irving,Strayer University-Katy,Strayer University-Northwest Houston,Strayer University-Plano,Strayer University-Cedar Hill,Strayer University-North Dallas,Strayer University-San Antonio,Strayer University-Stafford,WestMed College - Merced,Vantage College,SAE Institute of Technology San Francisco,Rasmussen College - Overland Park,National Personal Training Institute of Cleveland,Bay Area Medical Academy - San Jose Satellite Location,Excel Learning Center-San Antonio South
UGDS_WHITE,0.0333,0.5922,0.2990,0.6988,0.0158,0.7825,0.7255,0.7823,0.5328,0.8507,0.7983,0.4661,0.0280,0.3046,0.6408,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
UGDS_BLACK,0.9353,0.2600,0.4192,0.1255,0.9208,0.1119,0.2613,0.1200,0.3376,0.0704,0.1102,0.4372,0.8758,0.6054,0.2435,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
UGDS_HISP,0.0055,0.0283,0.0069,0.0382,0.0121,0.0348,0.0044,0.0191,0.0074,0.0248,0.0195,0.0492,0.0373,0.0153,0.0509,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
UGDS_ASIAN,0.0019,0.0518,0.0034,0.0376,0.0019,0.0106,0.0025,0.0053,0.0221,0.0227,0.0517,0.0127,0.0093,0.0153,0.0202,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
UGDS_AIAN,0.0024,0.0022,0.0000,0.0143,0.0010,0.0038,0.0044,0.0157,0.0044,0.0074,0.0102,0.0023,0.0000,0.0153,0.0081,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
UGDS_NHPI,0.0019,0.0007,0.0000,0.0002,0.0006,0.0009,0.0000,0.0010,0.0016,0.0000,0.0000,0.0035,0.0000,0.0096,0.0029,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
UGDS_2MOR,0.0000,0.0368,0.0000,0.0172,0.0098,0.0261,0.0000,0.0174,0.0297,0.0000,0.0051,0.0151,0.0031,0.0000,0.0254,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
UGDS_NRA,0.0059,0.0179,0.0000,0.0332,0.0243,0.0268,0.0000,0.0057,0.0397,0.0100,0.0000,0.0000,0.0466,0.0019,0.0012,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
UGDS_UNKN,0.0138,0.0100,0.2715,0.0350,0.0137,0.0026,0.0019,0.0334,0.0246,0.0140,0.0051,0.0139,0.0000,0.0326,0.0069,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Unstacking after a groupby aggregation
#### In this recipe, we use the employee dataset to perform an aggregation, grouping by multiple columns. We then use the unstack method to reshape the result into a format that makes for easier comparisons of different groups.

In [35]:
employee = pd.read_csv('master/data/employee.csv')
employee

,UNIQUE_ID,POSITION_TITLE,DEPARTMENT,BASE_SALARY,RACE,EMPLOYMENT_TYPE,GENDER,EMPLOYMENT_STATUS,HIRE_DATE,JOB_DATE
0,0,ASSISTANT DIRECTOR (EX LVL),Municipal Courts Department,121862.0,Hispanic/Latino,Full Time,Female,Active,2006-06-12,2012-10-13
1,1,LIBRARY ASSISTANT,Library,26125.0,Hispanic/Latino,Full Time,Female,Active,2000-07-19,2010-09-18
2,2,POLICE OFFICER,Houston Police Department-HPD,45279.0,White,Full Time,Male,Active,2015-02-03,2015-02-03
3,3,ENGINEER/OPERATOR,Houston Fire Department (HFD),63166.0,White,Full Time,Male,Active,1982-02-08,1991-05-25
4,4,ELECTRICIAN,General Services Department,56347.0,White,Full Time,Male,Active,1989-06-19,1994-10-22
5,5,SENIOR POLICE OFFICER,Houston Police Department-HPD,66614.0,Black or African American,Full Time,Male,Active,1984-11-26,2005-03-26
6,6,ENGINEER,Public Works & Engineering-PWE,71680.0,Asian/Pacific Islander,Full Time,Male,Active,2012-03-26,2012-03-26
7,7,CARPENTER,Houston Airport System (HAS),42390.0,White,Full Time,Male,Active,2013-11-04,2013-11-04
8,8,DEPUTY ASSISTANT DIRECTOR (EXECUTIVE LEV,Public Works & Engineering-PWE,107962.0,White,Full Time,Male,Active,1993-11-15,2013-01-05
9,9,AIRPORT OPERATIONS COORDINATOR,Houston Airport System (HAS),44616.0,NaN,Full Time,Male,Active,2016-03-14,2016-03-14


In [36]:
employee.groupby('RACE')['BASE_SALARY'].mean().astype(int)

RACE
American Indian or Alaskan Native    60272
Asian/Pacific Islander               61660
Black or African American            50137
Hispanic/Latino                      52345
Others                               51278
White                                64419
Name: BASE_SALARY, dtype: int32

In [37]:
agg = employee.groupby(['RACE', 'GENDER',])['BASE_SALARY'] \
              .mean().astype(int)

In [38]:
agg

RACE                               GENDER
American Indian or Alaskan Native  Female    60238
                                   Male      60305
Asian/Pacific Islander             Female    63226
                                   Male      61033
Black or African American          Female    48915
                                   Male      51082
Hispanic/Latino                    Female    46503
                                   Male      54782
Others                             Female    63785
                                   Male      38771
White                              Female    66793
                                   Male      63940
Name: BASE_SALARY, dtype: int32

In [39]:
agg.unstack('GENDER')

GENDER,Female,Male
RACE,,
American Indian or Alaskan Native,60238,60305
Asian/Pacific Islander,63226,61033
Black or African American,48915,51082
Hispanic/Latino,46503,54782
Others,63785,38771
White,66793,63940


In [40]:
agg.unstack('RACE')

RACE,American Indian or Alaskan Native,Asian/Pacific Islander,Black or African American,Hispanic/Latino,Others,White
GENDER,,,,,,
Female,60238,63226,48915,46503,63785,66793
Male,60305,61033,51082,54782,38771,63940


In [41]:
agg2 = employee.groupby(['RACE', 'GENDER'])['BASE_SALARY'] \
               .agg(['mean', 'max', 'min']).astype(int)
agg2

mean     max    min
RACE                              GENDER                      
American Indian or Alaskan Native Female  60238   98536  26125
                                  Male    60305   81239  26125
Asian/Pacific Islander            Female  63226  130416  26125
                                  Male    61033  163228  27914
Black or African American         Female  48915  150416  24960
                                  Male    51082  275000  26125
Hispanic/Latino                   Female  46503  126115  26125
                                  Male    54782  165216  26104
Others                            Female  63785   63785  63785
                                  Male    38771   38771  38771
White                             Female  66793  178331  27955
                                  Male    63940  210588  26125

In [42]:
agg2.unstack('GENDER')

mean            max            min       
GENDER                            Female   Male  Female    Male Female   Male
RACE                                                                         
American Indian or Alaskan Native  60238  60305   98536   81239  26125  26125
Asian/Pacific Islander             63226  61033  130416  163228  26125  27914
Black or African American          48915  51082  150416  275000  24960  26125
Hispanic/Latino                    46503  54782  126115  165216  26125  26104
Others                             63785  38771   63785   38771  63785  38771
White                              66793  63940  178331  210588  27955  26125

## Replicating pivot_table with a groupby aggregation
#### In this recipe, we use the flights dataset to create a pivot table and then recreate it using groupby operations.

In [43]:
flights = pd.read_csv('master/data/flights.csv')
fp = flights.pivot_table(index='AIRLINE',
                         columns='ORG_AIR',
                         values='CANCELLED',
                         aggfunc='sum',
                         fill_value=0).round(2)
fp.head()

ORG_AIR,ATL,DEN,DFW,IAH,LAS,LAX,MSP,ORD,PHX,SFO
AIRLINE,,,,,,,,,,
AA,3,4,86,3,3,11,3,35,4,2
AS,0,0,0,0,0,0,0,0,0,0
B6,0,0,0,0,0,0,0,0,0,1
DL,28,1,0,0,1,1,4,0,1,2
EV,18,6,27,36,0,0,6,53,0,0


In [44]:
fg = flights.groupby(['AIRLINE', 'ORG_AIR'])['CANCELLED'].sum()
fg.head()

AIRLINE  ORG_AIR
AA       ATL         3
         DEN         4
         DFW        86
         IAH         3
         LAS         3
Name: CANCELLED, dtype: int64

In [45]:
fg_unstack = fg.unstack('ORG_AIR', fill_value=0)
fp.equals(fg_unstack)

True

In [46]:
flights.pivot_table(index=['AIRLINE', 'MONTH'],
                    columns=['ORG_AIR', 'CANCELLED'],
                    values=['DEP_DELAY', 'DIST'],
                    aggfunc=[np.mean, np.sum],
                    fill_value=0).head()

mean                                                 \
              DEP_DELAY                                                  
ORG_AIR             ATL           DEN           DFW             IAH      
CANCELLED             0  1          0  1          0    1          0  1   
AIRLINE MONTH                                                            
AA      1     -3.250000  0   7.062500  0  11.977591 -3.0   9.750000  0   
        2     -3.000000  0   5.461538  0   8.756579  0.0   1.000000  0   
        3     -0.166667  0   7.666667  0  15.383784  0.0  10.900000  0   
        4      0.071429  0  20.266667  0  10.501493  0.0   6.933333  0   
        5      5.777778  0  23.466667  0  16.798780  0.0   3.055556  0   

                                                                   ...   \
                                                                   ...    
ORG_AIR              LAS          LAX           MSP           ORD  ...    
CANCELLED              0  1         0  1          0  1          0  ...    
AIRLINE MONTH                                                      ...    
AA      1      32.375000  0  5.617284  0  41.888889  0  15.702479  ...    
        2      -3.055556  0  3.123077  0   9.800000  0  11.277228  ...    
        3      12.074074  0  7.810811  0  -3.333333  0  18.633333  ...    
        4      27.241379  0  5.695652  0   0.142857  0   6.942446  ...    
        5       2.818182  0  4.119565  0   8.750000  0  12.018018  ...    

                 sum                                                           \
                DIST                                                            
ORG_AIR          DFW    IAH         LAS           LAX          MSP        ORD   
CANCELLED          1      0    1      0     1       0     1      0  1       0   
AIRLINE MONTH                                                                   
AA      1       7947   7024    0  45606     0  135921  2475   7281  0  129334   
        2      34049   3220  224  24112  2110  113483  5454   5040  0  120572   
        3      13044   5200    0  34313     0  131836  1744  14471  0  127072   
        4       3024  10020    0  28106     0  170285     0   4541  0  152154   
        5       5674   9212  224  47784     0  167484     0   6298  0  110864   

                                              
                                              
ORG_AIR                PHX         SFO        
CANCELLED         1      0    1      0     1  
AIRLINE MONTH                                 
AA      1         0  21018    0  33483     0  
        2      5398  17049  868  32110  2586  
        3       802  25770    0  43580     0  
        4      4718  17727    0  51054     0  
        5      1999  11164    0  40233     0  

[5 rows x 80 columns]

In [47]:
flights.groupby(['AIRLINE', 'MONTH', 'ORG_AIR', 'CANCELLED']) \
       ['DEP_DELAY', 'DIST'] \
       .agg(['mean', 'sum']) \
       .unstack(['ORG_AIR', 'CANCELLED'], fill_value=0) \
       .swaplevel(0, 1, axis='columns').head()

mean                                                    \
              DEP_DELAY                                                     
ORG_AIR             ATL            DEN            DFW             IAH       
CANCELLED             0   1          0   1          0    1          0   1   
AIRLINE MONTH                                                               
AA      1     -3.250000 NaN   7.062500 NaN  11.977591 -3.0   9.750000 NaN   
        2     -3.000000 NaN   5.461538 NaN   8.756579  NaN   1.000000 NaN   
        3     -0.166667 NaN   7.666667 NaN  15.383784  NaN  10.900000 NaN   
        4      0.071429 NaN  20.266667 NaN  10.501493  NaN   6.933333 NaN   
        5      5.777778 NaN  23.466667 NaN  16.798780  NaN   3.055556 NaN   

                                                                       ...    \
                                                                       ...     
ORG_AIR              LAS           LAX            MSP            ORD   ...     
CANCELLED              0   1         0   1          0   1          0   ...     
AIRLINE MONTH                                                          ...     
AA      1      32.375000 NaN  5.617284 NaN  41.888889 NaN  15.702479   ...     
        2      -3.055556 NaN  3.123077 NaN   9.800000 NaN  11.277228   ...     
        3      12.074074 NaN  7.810811 NaN  -3.333333 NaN  18.633333   ...     
        4      27.241379 NaN  5.695652 NaN   0.142857 NaN   6.942446   ...     
        5       2.818182 NaN  4.119565 NaN   8.750000 NaN  12.018018   ...     

                   sum                                                     \
                  DIST                                                      
ORG_AIR            DFW      IAH             LAS               LAX           
CANCELLED            1        0      1        0       1         0       1   
AIRLINE MONTH                                                               
AA      1       7947.0   7024.0    NaN  45606.0     NaN  135921.0  2475.0   
        2      34049.0   3220.0  224.0  24112.0  2110.0  113483.0  5454.0   
        3      13044.0   5200.0    NaN  34313.0     NaN  131836.0  1744.0   
        4       3024.0  10020.0    NaN  28106.0     NaN  170285.0     NaN   
        5       5674.0   9212.0  224.0  47784.0     NaN  167484.0     NaN   

                                                                               
                                                                               
ORG_AIR            MSP           ORD              PHX             SFO          
CANCELLED            0   1         0       1        0      1        0       1  
AIRLINE MONTH                                                                  
AA      1       7281.0 NaN  129334.0     NaN  21018.0    NaN  33483.0     NaN  
        2       5040.0 NaN  120572.0  5398.0  17049.0  868.0  32110.0  2586.0  
        3      14471.0 NaN  127072.0   802.0  25770.0    NaN  43580.0     NaN  
        4       4541.0 NaN  152154.0  4718.0  17727.0    NaN  51054.0     NaN  
        5       6298.0 NaN  110864.0  1999.0  11164.0    NaN  40233.0     NaN  

[5 rows x 80 columns]

## Renaming axis levels for easy reshaping
#### When grouping or aggregating with multiple columns, the resulting pandas object will have multiple levels in one or both of the axes. In this recipe, we will name each level of each axis and then use the methods stack/unstack to dramatically reshape the data to the desired form.

In [48]:
college = pd.read_csv('master/data/college.csv')
cg = college.groupby(['STABBR', 'RELAFFIL']) \
            ['UGDS', 'SATMTMID'] \
            .agg(['count', 'min', 'max']).head(6)
cg

UGDS                 SATMTMID              
                count    min      max    count    min    max
STABBR RELAFFIL                                             
AK     0            7  109.0  12865.0        0    NaN    NaN
       1            3   27.0    275.0        1  503.0  503.0
AL     0           71   12.0  29851.0       13  420.0  590.0
       1           18   13.0   3033.0        8  400.0  560.0
AR     0           68   18.0  21405.0        9  427.0  565.0
       1           14   20.0   4485.0        7  495.0  600.0

In [49]:
cg = cg.rename_axis(['AGG_COLS', 'AGG_FUNCS'], axis='columns')
cg

AGG_COLS         UGDS                 SATMTMID              
AGG_FUNCS       count    min      max    count    min    max
STABBR RELAFFIL                                             
AK     0            7  109.0  12865.0        0    NaN    NaN
       1            3   27.0    275.0        1  503.0  503.0
AL     0           71   12.0  29851.0       13  420.0  590.0
       1           18   13.0   3033.0        8  400.0  560.0
AR     0           68   18.0  21405.0        9  427.0  565.0
       1           14   20.0   4485.0        7  495.0  600.0

In [50]:
cg.stack('AGG_FUNCS').head()

AGG_COLS                      UGDS  SATMTMID
STABBR RELAFFIL AGG_FUNCS                   
AK     0        count          7.0       0.0
                min          109.0       NaN
                max        12865.0       NaN
       1        count          3.0       1.0
                min           27.0     503.0

In [51]:
cg.stack('AGG_FUNCS').swaplevel('AGG_FUNCS', 'STABBR',
                               axis='index').head()

,,AGG_COLS,UGDS,SATMTMID
AGG_FUNCS,RELAFFIL,STABBR,,
count,0,AK,7.0,0.0
min,0,AK,109.0,NaN
max,0,AK,12865.0,NaN
count,1,AK,3.0,1.0
min,1,AK,27.0,503.0


In [52]:
cg.stack('AGG_FUNCS') \
  .swaplevel('AGG_FUNCS', 'STABBR', axis='index') \
  .sort_index(level='RELAFFIL', axis='index') \
  .sort_index(level='AGG_COLS', axis='columns').head(6)

AGG_COLS                   SATMTMID     UGDS
AGG_FUNCS RELAFFIL STABBR                   
count     0        AK           0.0      7.0
                   AL          13.0     71.0
                   AR           9.0     68.0
max       0        AK           NaN  12865.0
                   AL         590.0  29851.0
                   AR         565.0  21405.0

In [53]:
cg.stack('AGG_FUNCS').unstack(['RELAFFIL', 'STABBR'])

AGG_COLS      UGDS                                          SATMTMID         \
RELAFFIL         0      1        0       1        0       1        0      1   
STABBR          AK     AK       AL      AL       AR      AR       AK     AK   
AGG_FUNCS                                                                     
count          7.0    3.0     71.0    18.0     68.0    14.0      0.0    1.0   
min          109.0   27.0     12.0    13.0     18.0    20.0      NaN  503.0   
max        12865.0  275.0  29851.0  3033.0  21405.0  4485.0      NaN  503.0   

AGG_COLS                               
RELAFFIL       0      1      0      1  
STABBR        AL     AL     AR     AR  
AGG_FUNCS                              
count       13.0    8.0    9.0    7.0  
min        420.0  400.0  427.0  495.0  
max        590.0  560.0  565.0  600.0

In [54]:
cg.stack(['AGG_FUNCS', 'AGG_COLS']).head(12)

STABBR  RELAFFIL  AGG_FUNCS  AGG_COLS
AK      0         count      UGDS            7.0
                             SATMTMID        0.0
                  min        UGDS          109.0
                  max        UGDS        12865.0
        1         count      UGDS            3.0
                             SATMTMID        1.0
                  min        UGDS           27.0
                             SATMTMID      503.0
                  max        UGDS          275.0
                             SATMTMID      503.0
AL      0         count      UGDS           71.0
                             SATMTMID       13.0
dtype: float64

In [55]:
cg.rename_axis([None, None], axis='index') \
  .rename_axis([None, None], axis='columns')

UGDS                 SATMTMID              
     count    min      max    count    min    max
AK 0     7  109.0  12865.0        0    NaN    NaN
   1     3   27.0    275.0        1  503.0  503.0
AL 0    71   12.0  29851.0       13  420.0  590.0
   1    18   13.0   3033.0        8  400.0  560.0
AR 0    68   18.0  21405.0        9  427.0  565.0
   1    14   20.0   4485.0        7  495.0  600.0

## Tidying when multiple variables are stored as column names
#### In this recipe, we will first identify all the variables of which some will be concatenated together as column names. We then reshape the data and parse the text to extract the correct variable values.

In [56]:
weightlifting = pd.read_csv('master/data/weightlifting_men.csv')
weightlifting

,Weight Category,M35 35-39,M40 40-44,M45 45-49,M50 50-54,M55 55-59,M60 60-64,M65 65-69,M70 70-74,M75 75-79,M80 80+
0,56,137,130,125,115,102,92,80,67,62,55
1,62,152,145,137,127,112,102,90,75,67,57
2,69,167,160,150,140,125,112,97,82,75,60
3,77,182,172,165,150,135,122,107,90,82,65
4,85,192,182,175,160,142,130,112,95,87,70
5,94,202,192,182,167,150,137,120,100,90,75
6,105,210,200,190,175,157,142,122,102,95,80
7,105+,217,207,197,182,165,150,127,107,100,85


In [57]:
wl_melt = weightlifting.melt(id_vars='Weight Category',
                             var_name='sex_age',
                             value_name='Qual Total')
wl_melt.head()

,Weight Category,sex_age,Qual Total
0,56,M35 35-39,137
1,62,M35 35-39,152
2,69,M35 35-39,167
3,77,M35 35-39,182
4,85,M35 35-39,192


In [58]:
sex_age = wl_melt['sex_age'].str.split(expand=True)
sex_age.head()

,0,1
0,M35,35-39
1,M35,35-39
2,M35,35-39
3,M35,35-39
4,M35,35-39


In [59]:
sex_age.columns = ['Sex', 'Age Group']
sex_age.head()

,Sex,Age Group
0,M35,35-39
1,M35,35-39
2,M35,35-39
3,M35,35-39
4,M35,35-39


In [60]:
sex_age['Sex'] = sex_age['Sex'].str[0]
sex_age.head()

,Sex,Age Group
0,M,35-39
1,M,35-39
2,M,35-39
3,M,35-39
4,M,35-39


In [61]:
wl_cat_total = wl_melt[['Weight Category', 'Qual Total']]
wl_tidy = pd.concat([sex_age, wl_cat_total], axis='columns')
wl_tidy.head()

,Sex,Age Group,Weight Category,Qual Total
0,M,35-39,56,137
1,M,35-39,62,152
2,M,35-39,69,167
3,M,35-39,77,182
4,M,35-39,85,192


In [62]:
cols = ['Weight Category', 'Qual Total']
sex_age[cols] = wl_melt[cols]
sex_age.head()

,Sex,Age Group,Weight Category,Qual Total
0,M,35-39,56,137
1,M,35-39,62,152
2,M,35-39,69,167
3,M,35-39,77,182
4,M,35-39,85,192


In [63]:
age_group = wl_melt.sex_age.str.extract('(\d{2}[+-](?:\d{2})?)',
                                        expand=False)
sex = wl_melt.sex_age.str[0]
new_cols = {'Sex':sex,
            'Age Group': age_group}
wl_tidy2 = wl_melt.assign(**new_cols) \
                  .drop('sex_age', axis='columns')

wl_tidy2.sort_index(axis=1).equals(wl_tidy.sort_index(axis=1))

True

## Tidying when multiple variables are stored as column values
#### In this recipe, we identify the column containing the improperly structured variables and pivot it to create tidy data.

In [64]:
inspections = pd.read_csv('master/data/restaurant_inspections.csv',
                          parse_dates=['Date'])
inspections.head()

,Name,Date,Info,Value
0,E & E Grill House,2017-08-08,Borough,MANHATTAN
1,E & E Grill House,2017-08-08,Cuisine,American
2,E & E Grill House,2017-08-08,Description,Non-food contact surface improperly constructe...
3,E & E Grill House,2017-08-08,Grade,A
4,E & E Grill House,2017-08-08,Score,9.0


In [65]:
# inspections.pivot(index=['Name', 'Date'],
#                   columns='Info', values='Value')
# NotImplementedError: > 1 ndim Categorical are not supported at this time

In [66]:
inspections.set_index(['Name', 'Date', 'Info']).head(10)

Value
Name              Date       Info                                                          
E & E Grill House 2017-08-08 Borough                                              MANHATTAN
                             Cuisine                                               American
                             Description  Non-food contact surface improperly constructe...
                             Grade                                                        A
                             Score                                                      9.0
PIZZA WAGON       2017-04-12 Borough                                               BROOKLYN
                             Cuisine                                                  Pizza
                             Description  Food contact surface not properly washed, rins...
                             Grade                                                        A
                             Score                                                     10.0

In [67]:
inspections.set_index(['Name', 'Date', 'Info']) \
           .unstack('Info').head()

Value  \
Info                                              Borough   
Name                                Date                    
3 STAR JUICE CENTER                 2017-05-10   BROOKLYN   
A & L PIZZA RESTAURANT              2017-08-22   BROOKLYN   
AKSARAY TURKISH CAFE AND RESTAURANT 2017-07-25   BROOKLYN   
ANTOJITOS DELI FOOD                 2017-06-01   BROOKLYN   
BANGIA                              2017-06-16  MANHATTAN   

                                                                                                   \
Info                                                                                      Cuisine   
Name                                Date                                                            
3 STAR JUICE CENTER                 2017-05-10                     Juice, Smoothies, Fruit Salads   
A & L PIZZA RESTAURANT              2017-08-22                                              Pizza   
AKSARAY TURKISH CAFE AND RESTAURANT 2017-07-25                                            Turkish   
ANTOJITOS DELI FOOD                 2017-06-01  Latin (Cuban, Dominican, Puerto Rican, South &...   
BANGIA                              2017-06-16                                             Korean   

                                                                                                   \
Info                                                                                  Description   
Name                                Date                                                            
3 STAR JUICE CENTER                 2017-05-10  Facility not vermin proof. Harborage or condit...   
A & L PIZZA RESTAURANT              2017-08-22  Facility not vermin proof. Harborage or condit...   
AKSARAY TURKISH CAFE AND RESTAURANT 2017-07-25  Plumbing not properly installed or maintained;...   
ANTOJITOS DELI FOOD                 2017-06-01  Live roaches present in facility's food and/or...   
BANGIA                              2017-06-16  Covered garbage receptacle not provided or ina...   

                                                            
Info                                           Grade Score  
Name                                Date                    
3 STAR JUICE CENTER                 2017-05-10     A  12.0  
A & L PIZZA RESTAURANT              2017-08-22     A   9.0  
AKSARAY TURKISH CAFE AND RESTAURANT 2017-07-25     A  13.0  
ANTOJITOS DELI FOOD                 2017-06-01     A  10.0  
BANGIA                              2017-06-16     A   9.0

In [68]:
insp_tidy = inspections.set_index(['Name', 'Date', 'Info']) \
                       .unstack('Info') \
                       .reset_index(col_level=-1)
insp_tidy.head()

Value  \
Info                                 Name       Date    Borough   
0                     3 STAR JUICE CENTER 2017-05-10   BROOKLYN   
1                  A & L PIZZA RESTAURANT 2017-08-22   BROOKLYN   
2     AKSARAY TURKISH CAFE AND RESTAURANT 2017-07-25   BROOKLYN   
3                     ANTOJITOS DELI FOOD 2017-06-01   BROOKLYN   
4                                  BANGIA 2017-06-16  MANHATTAN   

                                                         \
Info                                            Cuisine   
0                        Juice, Smoothies, Fruit Salads   
1                                                 Pizza   
2                                               Turkish   
3     Latin (Cuban, Dominican, Puerto Rican, South &...   
4                                                Korean   

                                                                     
Info                                        Description Grade Score  
0     Facility not vermin proof. Harborage or condit...     A  12.0  
1     Facility not vermin proof. Harborage or condit...     A   9.0  
2     Plumbing not properly installed or maintained;...     A  13.0  
3     Live roaches present in facility's food and/or...     A  10.0  
4     Covered garbage receptacle not provided or ina...     A   9.0

In [69]:
insp_tidy.columns = insp_tidy.columns.droplevel(0) \
                             .rename(None)
insp_tidy.head()

,Name,Date,Borough,Cuisine,Description,Grade,Score
0,3 STAR JUICE CENTER,2017-05-10,BROOKLYN,"Juice, Smoothies, Fruit Salads",Facility not vermin proof. Harborage or condit...,A,12.0
1,A & L PIZZA RESTAURANT,2017-08-22,BROOKLYN,Pizza,Facility not vermin proof. Harborage or condit...,A,9.0
2,AKSARAY TURKISH CAFE AND RESTAURANT,2017-07-25,BROOKLYN,Turkish,Plumbing not properly installed or maintained;...,A,13.0
3,ANTOJITOS DELI FOOD,2017-06-01,BROOKLYN,"Latin (Cuban, Dominican, Puerto Rican, South &...",Live roaches present in facility's food and/or...,A,10.0
4,BANGIA,2017-06-16,MANHATTAN,Korean,Covered garbage receptacle not provided or ina...,A,9.0


In [70]:
inspections.set_index(['Name', 'Date', 'Info']) \
           .squeeze() \
           .unstack('Info') \
           .reset_index() \
           .rename_axis(None, axis='columns')

,Name,Date,Borough,Cuisine,Description,Grade,Score
0,3 STAR JUICE CENTER,2017-05-10,BROOKLYN,"Juice, Smoothies, Fruit Salads",Facility not vermin proof. Harborage or condit...,A,12.0
1,A & L PIZZA RESTAURANT,2017-08-22,BROOKLYN,Pizza,Facility not vermin proof. Harborage or condit...,A,9.0
2,AKSARAY TURKISH CAFE AND RESTAURANT,2017-07-25,BROOKLYN,Turkish,Plumbing not properly installed or maintained;...,A,13.0
3,ANTOJITOS DELI FOOD,2017-06-01,BROOKLYN,"Latin (Cuban, Dominican, Puerto Rican, South &...",Live roaches present in facility's food and/or...,A,10.0
4,BANGIA,2017-06-16,MANHATTAN,Korean,Covered garbage receptacle not provided or ina...,A,9.0
5,BANGKOK CUISINE,2017-07-19,MANHATTAN,Thai,Non-food contact surface improperly constructe...,A,13.0
6,BASIL,2017-05-03,BROOKLYN,Jewish/Kosher,Cold food item held above 41Âº F (smoked fish ...,A,13.0
7,BEIT JEDDO,2017-03-23,BROOKLYN,Middle Eastern,Thawing procedures improper.,A,13.0
8,BIG FLEISHIG'S EXPRESS,2017-02-22,BROOKLYN,Jewish/Kosher,"Single service item reused, improperly stored,...",A,12.0
9,BLOSSOM ON COLUMBUS,2017-01-25,MANHATTAN,American,"Food contact surface not properly washed, rins...",A,10.0


In [71]:
inspections.pivot_table(index=['Name', 'Date'],
                        columns='Info',
                        values='Value',
                        aggfunc='first') \
           .reset_index() \
           .rename_axis(None, axis='columns')

,Name,Date,Borough,Cuisine,Description,Grade,Score
0,3 STAR JUICE CENTER,2017-05-10,BROOKLYN,"Juice, Smoothies, Fruit Salads",Facility not vermin proof. Harborage or condit...,A,12.0
1,A & L PIZZA RESTAURANT,2017-08-22,BROOKLYN,Pizza,Facility not vermin proof. Harborage or condit...,A,9.0
2,AKSARAY TURKISH CAFE AND RESTAURANT,2017-07-25,BROOKLYN,Turkish,Plumbing not properly installed or maintained;...,A,13.0
3,ANTOJITOS DELI FOOD,2017-06-01,BROOKLYN,"Latin (Cuban, Dominican, Puerto Rican, South &...",Live roaches present in facility's food and/or...,A,10.0
4,BANGIA,2017-06-16,MANHATTAN,Korean,Covered garbage receptacle not provided or ina...,A,9.0
5,BANGKOK CUISINE,2017-07-19,MANHATTAN,Thai,Non-food contact surface improperly constructe...,A,13.0
6,BASIL,2017-05-03,BROOKLYN,Jewish/Kosher,Cold food item held above 41Âº F (smoked fish ...,A,13.0
7,BEIT JEDDO,2017-03-23,BROOKLYN,Middle Eastern,Thawing procedures improper.,A,13.0
8,BIG FLEISHIG'S EXPRESS,2017-02-22,BROOKLYN,Jewish/Kosher,"Single service item reused, improperly stored,...",A,12.0
9,BLOSSOM ON COLUMBUS,2017-01-25,MANHATTAN,American,"Food contact surface not properly washed, rins...",A,10.0


## Tidying when two or more values are stored in the same cell
#### In this recipe, we examine a dataset that has a column containing multiple different variables in each cell. We use the str accessor to parse these strings into separate columns to tidy the data.

In [72]:
cities = pd.read_csv('master/data/texas_cities.csv')
cities

,City,Geolocation
0,Houston,"29.7604° N, 95.3698° W"
1,Dallas,"32.7767° N, 96.7970° W"
2,Austin,"30.2672° N, 97.7431° W"


In [73]:
geolocations = cities.Geolocation.str.split(pat='. ',
                                             expand=True)
geolocations.columns = ['latitude', 'latitude direction',
                        'longitude', 'longitude direction']
geolocations

,latitude,latitude direction,longitude,longitude direction
0,29.7604,N,95.3698,W
1,32.7767,N,96.7970,W
2,30.2672,N,97.7431,W


In [74]:
geolocations = geolocations.astype({'latitude':'float',
                                    'longitude':'float'})
geolocations.dtypes

latitude               float64
latitude direction      object
longitude              float64
longitude direction     object
dtype: object

In [75]:
cities_tidy = pd.concat([cities['City'], geolocations],
                        axis='columns')
cities_tidy

,City,latitude,latitude direction,longitude,longitude direction
0,Houston,29.7604,N,95.3698,W
1,Dallas,32.7767,N,96.7970,W
2,Austin,30.2672,N,97.7431,W


In [76]:
geolocations.apply(pd.to_numeric, errors='ignore')

,latitude,latitude direction,longitude,longitude direction
0,29.7604,N,95.3698,W
1,32.7767,N,96.7970,W
2,30.2672,N,97.7431,W


In [77]:
cities.Geolocation.str.split('° |, ', expand=True)

,0,1,2,3
0,29.7604,N,95.3698,W
1,32.7767,N,96.7970,W
2,30.2672,N,97.7431,W


In [78]:
cities.Geolocation.str.extract('([0-9.]+). (N|S), ([0-9.]+). (E|W)',
                               expand=True)

,0,1,2,3
0,29.7604,N,95.3698,W
1,32.7767,N,96.7970,W
2,30.2672,N,97.7431,W


## Tidying when variables are stored in column names and values
#### In this recipe, variables are identified both vertically and horizontally and reshaped into tidy data with the melt and pivot_table methods.

In [79]:
sensors = pd.read_csv('master/data/sensors.csv')
sensors

,Group,Property,2012,2013,2014,2015,2016
0,A,Pressure,928,873,814,973,870
1,A,Temperature,1026,1038,1009,1036,1042
2,A,Flow,819,806,861,882,856
3,B,Pressure,817,877,914,806,942
4,B,Temperature,1008,1041,1009,1002,1013
5,B,Flow,887,899,837,824,873


In [80]:
sensors.melt(id_vars=['Group', 'Property'], var_name='Year') \
       .head(6)

,Group,Property,Year,value
0,A,Pressure,2012,928
1,A,Temperature,2012,1026
2,A,Flow,2012,819
3,B,Pressure,2012,817
4,B,Temperature,2012,1008
5,B,Flow,2012,887


In [82]:
sensors.melt(id_vars=['Group', 'Property'], var_name='Year') \
       .pivot_table(index=['Group', 'Year'],
                    columns='Property', values='value') \
       .reset_index() \
       .rename_axis(None, axis='columns')

,Group,Year,Flow,Pressure,Temperature
0,A,2012,819,928,1026
1,A,2013,806,873,1038
2,A,2014,861,814,1009
3,A,2015,882,973,1036
4,A,2016,856,870,1042
5,B,2012,887,817,1008
6,B,2013,899,877,1041
7,B,2014,837,914,1009
8,B,2015,824,806,1002
9,B,2016,873,942,1013


In [83]:
sensors.set_index(['Group', 'Property']) \
       .stack() \
       .unstack('Property') \
       .rename_axis(['Group', 'Year'], axis='index') \
       .rename_axis(None, axis='columns') \
       .reset_index()

,Group,Year,Flow,Pressure,Temperature
0,A,2012,819,928,1026
1,A,2013,806,873,1038
2,A,2014,861,814,1009
3,A,2015,882,973,1036
4,A,2016,856,870,1042
5,B,2012,887,817,1008
6,B,2013,899,877,1041
7,B,2014,837,914,1009
8,B,2015,824,806,1002
9,B,2016,873,942,1013


## Tidying when multiple observational units are stored in the same table
#### In this recipe, we use the movie dataset to identify the three observational units (movies, actors, and directors) and create separate tables for each. One of the keys to this recipe is understanding that the actor and director Facebook likes are independent of the movie. Each actor and director is mapped to a single value representing their number of Facebook likes. Due to this independence, we can separate the data for the movies, directors, and actors into their own tables. Database folks call this process normalization, which increases data integrity and reduces redundancy.

In [106]:
movie = pd.read_csv('master/data/movie_altered.csv')
movie.head()

,title,rating,year,duration,director_1,director_fb_likes_1,actor_1,actor_2,actor_3,actor_fb_likes_1,actor_fb_likes_2,actor_fb_likes_3
0,Avatar,PG-13,2009.0,178.0,James Cameron,0.0,CCH Pounder,Joel David Moore,Wes Studi,1000.0,936.0,855.0
1,Pirates of the Caribbean: At World's End,PG-13,2007.0,169.0,Gore Verbinski,563.0,Johnny Depp,Orlando Bloom,Jack Davenport,40000.0,5000.0,1000.0
2,Spectre,PG-13,2015.0,148.0,Sam Mendes,0.0,Christoph Waltz,Rory Kinnear,Stephanie Sigman,11000.0,393.0,161.0
3,The Dark Knight Rises,PG-13,2012.0,164.0,Christopher Nolan,22000.0,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,27000.0,23000.0,23000.0
4,Star Wars: Episode VII - The Force Awakens,NaN,NaN,NaN,Doug Walker,131.0,Doug Walker,Rob Walker,NaN,131.0,12.0,NaN


In [107]:
movie.insert(0, 'id', np.arange(len(movie)))
movie.head()

,id,title,rating,year,duration,director_1,director_fb_likes_1,actor_1,actor_2,actor_3,actor_fb_likes_1,actor_fb_likes_2,actor_fb_likes_3
0,0,Avatar,PG-13,2009.0,178.0,James Cameron,0.0,CCH Pounder,Joel David Moore,Wes Studi,1000.0,936.0,855.0
1,1,Pirates of the Caribbean: At World's End,PG-13,2007.0,169.0,Gore Verbinski,563.0,Johnny Depp,Orlando Bloom,Jack Davenport,40000.0,5000.0,1000.0
2,2,Spectre,PG-13,2015.0,148.0,Sam Mendes,0.0,Christoph Waltz,Rory Kinnear,Stephanie Sigman,11000.0,393.0,161.0
3,3,The Dark Knight Rises,PG-13,2012.0,164.0,Christopher Nolan,22000.0,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,27000.0,23000.0,23000.0
4,4,Star Wars: Episode VII - The Force Awakens,NaN,NaN,NaN,Doug Walker,131.0,Doug Walker,Rob Walker,NaN,131.0,12.0,NaN


In [108]:
stubnames = ['director', 'director_fb_likes',
             'actor', 'actor_fb_likes']
movie_long = pd.wide_to_long(movie,
                             stubnames=stubnames,
                             i='id',
                             j='num',
                             sep='_').reset_index()

In [109]:
movie_long['num'] = movie_long['num'].astype(int)
movie_long.head()

,id,num,rating,duration,title,year,director,director_fb_likes,actor,actor_fb_likes
0,0,1,PG-13,178.0,Avatar,2009.0,James Cameron,0.0,CCH Pounder,1000.0
1,0,2,PG-13,178.0,Avatar,2009.0,NaN,NaN,Joel David Moore,936.0
2,0,3,PG-13,178.0,Avatar,2009.0,NaN,NaN,Wes Studi,855.0
3,1,1,PG-13,169.0,Pirates of the Caribbean: At World's End,2007.0,Gore Verbinski,563.0,Johnny Depp,40000.0
4,1,2,PG-13,169.0,Pirates of the Caribbean: At World's End,2007.0,NaN,NaN,Orlando Bloom,5000.0


In [117]:
movie_table = movie_long[['id', 'title', 'year', 'duration', 'rating']]
movie_table.head(11)

,id,title,year,duration,rating
0,0,Avatar,2009.0,178.0,PG-13
1,0,Avatar,2009.0,178.0,PG-13
2,0,Avatar,2009.0,178.0,PG-13
3,1,Pirates of the Caribbean: At World's End,2007.0,169.0,PG-13
4,1,Pirates of the Caribbean: At World's End,2007.0,169.0,PG-13
5,1,Pirates of the Caribbean: At World's End,2007.0,169.0,PG-13
6,2,Spectre,2015.0,148.0,PG-13
7,2,Spectre,2015.0,148.0,PG-13
8,2,Spectre,2015.0,148.0,PG-13
9,3,The Dark Knight Rises,2012.0,164.0,PG-13


In [118]:
director_table = movie_long[['id', 'num',
                             'director', 'director_fb_likes']]
director_table.head(11)

,id,num,director,director_fb_likes
0,0,1,James Cameron,0.0
1,0,2,NaN,NaN
2,0,3,NaN,NaN
3,1,1,Gore Verbinski,563.0
4,1,2,NaN,NaN
5,1,3,NaN,NaN
6,2,1,Sam Mendes,0.0
7,2,2,NaN,NaN
8,2,3,NaN,NaN
9,3,1,Christopher Nolan,22000.0


In [128]:
actor_table = movie_long[['id', 'num',
                          'actor', 'actor_fb_likes']]
actor_table.head(11)

,id,num,actor,actor_fb_likes
0,0,1,CCH Pounder,1000.0
1,0,2,Joel David Moore,936.0
2,0,3,Wes Studi,855.0
3,1,1,Johnny Depp,40000.0
4,1,2,Orlando Bloom,5000.0
5,1,3,Jack Davenport,1000.0
6,2,1,Christoph Waltz,11000.0
7,2,2,Rory Kinnear,393.0
8,2,3,Stephanie Sigman,161.0
9,3,1,Tom Hardy,27000.0


In [129]:
movie_table = movie_table.drop_duplicates() \
                         .reset_index(drop=True)
movie_table.head()

,id,title,year,duration,rating
0,0,Avatar,2009.0,178.0,PG-13
1,1,Pirates of the Caribbean: At World's End,2007.0,169.0,PG-13
2,2,Spectre,2015.0,148.0,PG-13
3,3,The Dark Knight Rises,2012.0,164.0,PG-13
4,4,Star Wars: Episode VII - The Force Awakens,NaN,NaN,NaN


In [130]:
director_table = director_table.dropna() \
                               .reset_index(drop=True)
director_table.head()

,id,num,director,director_fb_likes
0,0,1,James Cameron,0.0
1,1,1,Gore Verbinski,563.0
2,2,1,Sam Mendes,0.0
3,3,1,Christopher Nolan,22000.0
4,4,1,Doug Walker,131.0


In [131]:
actor_table = actor_table.dropna() \
                         .reset_index(drop=True)
actor_table.head()

,id,num,actor,actor_fb_likes
0,0,1,CCH Pounder,1000.0
1,0,2,Joel David Moore,936.0
2,0,3,Wes Studi,855.0
3,1,1,Johnny Depp,40000.0
4,1,2,Orlando Bloom,5000.0


In [132]:
movie.memory_usage(deep=True).sum()

2289770

In [133]:
movie_table.memory_usage(deep=True).sum()+ \
director_table.memory_usage(deep=True).sum()+ \
actor_table.memory_usage(deep=True).sum()

2538022

In [134]:
# we need to create another table mapping each movie to each actor/director.
# Let's first create id columns specific to the actor and director tables, uniquely identifying each actor/director:

director_cat = pd.Categorical(director_table['director'])
director_table.insert(1, 'director_id', director_cat.codes)

actor_cat = pd.Categorical(actor_table['actor'])
actor_table.insert(1, 'actor_id', actor_cat.codes)

In [135]:
director_associative = director_table[['id', 'director_id',
                                       'num']]
director_associative.head()

,id,director_id,num
0,0,922,1
1,1,794,1
2,2,2020,1
3,3,373,1
4,4,600,1


In [137]:
dcols = ['director_id', 'director', 'director_fb_likes']
director_unique = director_table[dcols].drop_duplicates() \
                                       .reset_index(drop=True)
director_unique.head()

,director_id,director,director_fb_likes
0,922,James Cameron,0.0
1,794,Gore Verbinski,563.0
2,2020,Sam Mendes,0.0
3,373,Christopher Nolan,22000.0
4,600,Doug Walker,131.0


In [138]:
actor_associative = actor_table[['id', 'actor_id',
                                       'num']]
actor_associative.head()

,id,actor_id,num
0,0,824,1
1,0,2867,2
2,0,6099,3
3,1,2971,1
4,1,4536,2


In [140]:
acols = ['actor_id', 'actor', 'actor_fb_likes']
actor_unique = actor_table[acols].drop_duplicates() \
                                       .reset_index(drop=True)
actor_unique.head()

,actor_id,actor,actor_fb_likes
0,824,CCH Pounder,1000.0
1,2867,Joel David Moore,936.0
2,6099,Wes Studi,855.0
3,2971,Johnny Depp,40000.0
4,4536,Orlando Bloom,5000.0


In [141]:
movie_table.memory_usage(deep=True).sum()+ \
director_associative.memory_usage(deep=True).sum()+ \
director_unique.memory_usage(deep=True).sum()+ \
actor_associative.memory_usage(deep=True).sum()+ \
actor_unique.memory_usage(deep=True).sum()

1746526

In [149]:
# It is possible to recreate the original movie table by joining all the tables back together.
# First, join the associative tables to the actor/director tables.
# Then pivot the num column, and add the column prefixes back:

actors = actor_associative.merge(actor_unique, on='actor_id') \
                          .drop('actor_id', 1) \
                          .pivot_table(index='id',
                                       columns='num',
                                       aggfunc='first')

In [150]:
actors.columns = actors.columns.get_level_values(0) + '_' + \
                 actors.columns.get_level_values(1).astype(str)

In [151]:
directors = director_associative.merge(director_unique,
                                       on='director_id') \
                                .drop('director_id', 1) \
                                .pivot_table(index='id',
                                             columns='num',
                                             aggfunc='first')

In [152]:
directors.columns = directors.columns.get_level_values(0) + '_' + \
                    directors.columns.get_level_values(1).astype(str)

In [153]:
movie2 = movie_table.merge(directors.reset_index(),
                           on='id', how='left') \
                    .merge(actors.reset_index(),
                           on='id', how='left')

In [155]:
movie.head()

,id,title,rating,year,duration,director_1,director_fb_likes_1,actor_1,actor_2,actor_3,actor_fb_likes_1,actor_fb_likes_2,actor_fb_likes_3
0,0,Avatar,PG-13,2009.0,178.0,James Cameron,0.0,CCH Pounder,Joel David Moore,Wes Studi,1000.0,936.0,855.0
1,1,Pirates of the Caribbean: At World's End,PG-13,2007.0,169.0,Gore Verbinski,563.0,Johnny Depp,Orlando Bloom,Jack Davenport,40000.0,5000.0,1000.0
2,2,Spectre,PG-13,2015.0,148.0,Sam Mendes,0.0,Christoph Waltz,Rory Kinnear,Stephanie Sigman,11000.0,393.0,161.0
3,3,The Dark Knight Rises,PG-13,2012.0,164.0,Christopher Nolan,22000.0,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,27000.0,23000.0,23000.0
4,4,Star Wars: Episode VII - The Force Awakens,NaN,NaN,NaN,Doug Walker,131.0,Doug Walker,Rob Walker,NaN,131.0,12.0,NaN


In [156]:
movie2[movie.columns].head()

,id,title,rating,year,duration,director_1,director_fb_likes_1,actor_1,actor_2,actor_3,actor_fb_likes_1,actor_fb_likes_2,actor_fb_likes_3
0,0,Avatar,PG-13,2009.0,178.0,James Cameron,0.0,CCH Pounder,Joel David Moore,Wes Studi,1000.0,936.0,855.0
1,1,Pirates of the Caribbean: At World's End,PG-13,2007.0,169.0,Gore Verbinski,563.0,Johnny Depp,Orlando Bloom,Jack Davenport,40000.0,5000.0,1000.0
2,2,Spectre,PG-13,2015.0,148.0,Sam Mendes,0.0,Christoph Waltz,Rory Kinnear,Stephanie Sigman,11000.0,393.0,161.0
3,3,The Dark Knight Rises,PG-13,2012.0,164.0,Christopher Nolan,22000.0,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,27000.0,23000.0,23000.0
4,4,Star Wars: Episode VII - The Force Awakens,NaN,NaN,NaN,Doug Walker,131.0,Doug Walker,Rob Walker,NaN,131.0,12.0,NaN


In [165]:
movie.head().equals(movie2[movie.columns].head())

False

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\John\Anaconda3\lib\asyncio\events.py", line 88, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\John\Anaconda3\lib\asyncio\selector_events.py", line 125, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] Une connexion existante a dû être fermée par l’hôte distant
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\John\Anaconda3\lib\asyncio\events.py", line 88, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\John\Anaconda3\lib\asyncio\selector_events.py", line 125, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] Une connexion existante a dû être fermée par l’hôt

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\John\Anaconda3\lib\asyncio\events.py", line 88, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\John\Anaconda3\lib\asyncio\selector_events.py", line 125, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] Une connexion existante a dû être fermée par l’hôte distant
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\John\Anaconda3\lib\asyncio\events.py", line 88, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\John\Anaconda3\lib\asyncio\selector_events.py", line 125, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] Une connexion existante a dû être fermée par l’hôt

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\John\Anaconda3\lib\asyncio\events.py", line 88, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\John\Anaconda3\lib\asyncio\selector_events.py", line 125, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] Une connexion existante a dû être fermée par l’hôte distant
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\John\Anaconda3\lib\asyncio\events.py", line 88, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\John\Anaconda3\lib\asyncio\selector_events.py", line 125, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] Une connexion existante a dû être fermée par l’hôt

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\John\Anaconda3\lib\asyncio\events.py", line 88, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\John\Anaconda3\lib\asyncio\selector_events.py", line 125, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] Une connexion existante a dû être fermée par l’hôte distant
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\John\Anaconda3\lib\asyncio\events.py", line 88, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\John\Anaconda3\lib\asyncio\selector_events.py", line 125, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] Une connexion existante a dû être fermée par l’hôt

...

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\John\Anaconda3\lib\asyncio\events.py", line 88, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\John\Anaconda3\lib\asyncio\selector_events.py", line 125, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] Une connexion existante a dû être fermée par l’hôte distant
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\John\Anaconda3\lib\asyncio\events.py", line 88, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\John\Anaconda3\lib\asyncio\selector_events.py", line 125, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] Une connexion existante a dû être fermée par l’hôt

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\John\Anaconda3\lib\asyncio\events.py", line 88, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\John\Anaconda3\lib\asyncio\selector_events.py", line 125, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] Une connexion existante a dû être fermée par l’hôte distant
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\John\Anaconda3\lib\asyncio\events.py", line 88, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\John\Anaconda3\lib\asyncio\selector_events.py", line 125, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] Une connexion existante a dû être fermée par l’hôt

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\John\Anaconda3\lib\asyncio\events.py", line 88, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\John\Anaconda3\lib\asyncio\selector_events.py", line 125, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] Une connexion existante a dû être fermée par l’hôte distant
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\John\Anaconda3\lib\asyncio\events.py", line 88, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\John\Anaconda3\lib\asyncio\selector_events.py", line 125, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] Une connexion existante a dû être fermée par l’hôt

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\John\Anaconda3\lib\asyncio\events.py", line 88, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\John\Anaconda3\lib\asyncio\selector_events.py", line 125, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] Une connexion existante a dû être fermée par l’hôte distant
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\John\Anaconda3\lib\asyncio\events.py", line 88, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\John\Anaconda3\lib\asyncio\selector_events.py", line 125, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] Une connexion existante a dû être fermée par l’hôt

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\John\Anaconda3\lib\asyncio\events.py", line 88, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\John\Anaconda3\lib\asyncio\selector_events.py", line 125, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] Une connexion existante a dû être fermée par l’hôte distant
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\John\Anaconda3\lib\asyncio\events.py", line 88, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\John\Anaconda3\lib\asyncio\selector_events.py", line 125, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] Une connexion existante a dû être fermée par l’hôt